In [2]:
!pip install "git+https://github.com/facebookresearch/pytorch3d.git"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/pytorch3d.git to /tmp/pip-req-build-htw_d414
  Running command git clone -q https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-htw_d414
     |████████████████████████████████| 50 kB 2.5 MB/s 
     |████████████████████████████████| 596 kB 8.5 MB/s 
  Created wheel for pytorch3d: filename=pytorch3d-0.6.2-cp37-cp37m-linux_x86_64.whl size=31520148 sha256=53cc5a3d40fe77b203cbbe3cce7f4ad45a793780881f58ba4fea8c2c46dcd4ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-ij0tqhzy/wheels/db/2f/07/b84807ee4c9ffc917b90b716977d7fea8f9e841f6173150600
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=e53b438a9c0e43d57978d217435ad755f04b1707528cc474e692bbb3a0fd9266
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
Successfully built pyt

In [3]:
import os
import numpy as np
import json

from PIL import Image  
from csv import writer
import csv

from tensorflow import keras
import tensorflow as tf

from typing import Tuple

import torch
import torch.nn.functional as F
from pytorch3d import _C
from torch.autograd import Function

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jun  6 07:54:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    33W / 250W |    629MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
WIDTH = 730
HEIGHT = 530
CHANNELS = 3

BATCH_SIZE = 8
TRAIN_DATASET_NO = 3136

TEST_DATASET_NO = 348

SAMPLES_NO = TRAIN_DATASET_NO // BATCH_SIZE

DATA_PATH = "datasets/SUNRGB-D3D/SUNRGBD/kv2/kinect2data"
CSV_EXISTS = True


X_TRAIN_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/train_images_k2.csv'
y_TRAIN_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/train_annotations_k2.csv'
X_TEST_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/test_images_k2.csv'
y_TEST_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/test_annotations_k2.csv'
X_PREDICT_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/predict_k2.csv'


In [ ]:
def csv_lazy(file_name):
    with open(file_name) as f:
        r = csv.reader(f)
        
        for row in r:
            yield row

def batch_generator(X_file, batch_size):

    csv_generator = csv_lazy(X_file)

    while True:
        batch = np.array([])

        for i in range(batch_size):
            batch = np.append(batch, np.array(next(csv_generator)).astype('float') / 255.0)
        
        yield batch.reshape(batch_size, WIDTH, HEIGHT, CHANNELS).astype('float')

def label_generator(y_file, batch_size):
    csv_generator = csv_lazy(y_file)

    while True:
        batch = np.array([])

        for i in range(batch_size):
            batch = np.append(batch, next(csv_generator))
        
        yield batch.reshape(batch_size, 10).astype('float')
        
def model_generator(X_file, y_file, batch_size, limit_iteration):
    X_generator = batch_generator(X_file, batch_size)
    y_generator = label_generator(y_file, batch_size)
    
    iteration_no = 0

    while True:
        
        iteration_no += 1

        print('Iteration: ' + str(iteration_no))

        # Reset to start of file
        if iteration_no > limit_iteration:
            X_generator = batch_generator(X_file, batch_size)
            y_generator = label_generator(y_file, batch_size)
            iteration_no = 0

        X_batch = next(X_generator)
        y_batch = next(y_generator)

        yield (X_batch, y_batch)

In [ ]:
base_model = keras.applications.xception.Xception(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
loc_output = keras.layers.Dense(10)(avg)

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)

model = keras.models.Model(inputs=base_model.input, outputs=loc_output)
model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

83697664/83683744 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
SAMPLES_NO

392

In [ ]:
model.fit(x = model_generator(X_TRAIN_DATA_PATH, y_TRAIN_DATA_PATH, BATCH_SIZE, SAMPLES_NO // 10),
          steps_per_epoch = SAMPLES_NO // 10,
          epochs = 10,
          verbose = 2
          )

Iteration: 1
Here
Epoch 1/10
Iteration: 2
Here
Iteration: 3
Here
Iteration: 4
Here
Iteration: 5
Here
Iteration: 6
Here
Iteration: 7
Here
Iteration: 8
Here
Iteration: 9
Here
Iteration: 10
Here
Iteration: 11
Here
Iteration: 12
Here
Iteration: 13
Here
Iteration: 14
Here
Iteration: 15
Here
Iteration: 16
Here
Iteration: 17
Here
Iteration: 18
Here
Iteration: 19
Here
Iteration: 20
Here
Iteration: 21
Here
Iteration: 22
Here
Iteration: 23
Here
Iteration: 24
Here
Iteration: 25
Here
Iteration: 26
Here
Iteration: 27
Here
Iteration: 28
Here
Iteration: 29
Here
Iteration: 30
Here
Iteration: 31
Here
Iteration: 32
Here
Iteration: 33
Here
Iteration: 34
Here
Iteration: 35
Here
Iteration: 36
Here
Iteration: 37
Here
Iteration: 38
Here
Iteration: 39
Here
Iteration: 40
Here
39/39 - 204s - loss: 0.7539 - accuracy: 0.5160 - 204s/epoch - 5s/step
Epoch 2/10
Iteration: 1
Here
Iteration: 2
Here
Iteration: 3
Here
Iteration: 4
Here
Iteration: 5
Here
Iteration: 6
Here
Iteration: 7
Here
Iteration: 8
Here
Iteration: 9


In [ ]:
TEST_DATASET_NO // 4

87

In [ ]:
model.evaluate(
    x = model_generator(X_TEST_DATA_PATH, y_TEST_DATA_PATH, 4, TEST_DATASET_NO // 4),
    verbose = 2,
    steps = TEST_DATASET_NO // 4 - 1
)

Iteration: 1
Here
Iteration: 2
Here
Iteration: 3
Here
Iteration: 4
Here
Iteration: 5
Here
Iteration: 6
Here
Iteration: 7
Here
Iteration: 8
Here
Iteration: 9
Here
Iteration: 10
Here
Iteration: 11
Here
Iteration: 12
Here
Iteration: 13
Here
Iteration: 14
Here
Iteration: 15
Here
Iteration: 16
Here
Iteration: 17
Here
Iteration: 18
Here
Iteration: 19
Here
Iteration: 20
Here
Iteration: 21
Here
Iteration: 22
Here
Iteration: 23
Here
Iteration: 24
Here
Iteration: 25
Here
Iteration: 26
Here
Iteration: 27
Here
Iteration: 28
Here
Iteration: 29
Here
Iteration: 30
Here
Iteration: 31
Here
Iteration: 32
Here
Iteration: 33
Here
Iteration: 34
Here
Iteration: 35
Here
Iteration: 36
Here
Iteration: 37
Here
Iteration: 38
Here
Iteration: 39
Here
Iteration: 40
Here
Iteration: 41
Here
Iteration: 42
Here
Iteration: 43
Here
Iteration: 44
Here
Iteration: 45
Here
Iteration: 46
Here
Iteration: 47
Here
Iteration: 48
Here
Iteration: 49
Here
Iteration: 50
Here
Iteration: 51
Here
Iteration: 52
Here
Iteration: 53
Here
It

[0.6675821542739868, 0.2005814015865326]

In [ ]:
base_model = keras.applications.xception.Xception(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
loc_output = keras.layers.Dense(10)(avg)

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)

model = keras.models.Model(inputs=base_model.input, outputs=loc_output)
model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
model.fit(x = model_generator(X_TRAIN_DATA_PATH, y_TRAIN_DATA_PATH, BATCH_SIZE, SAMPLES_NO // 4),
          steps_per_epoch = SAMPLES_NO // 4,
          epochs = 10,
          verbose = 2
          )

Iteration: 1
Here
Epoch 1/10
Iteration: 2
Here
Iteration: 3
Here
Iteration: 4
Here
Iteration: 5
Here
Iteration: 6
Here
Iteration: 7
Here
Iteration: 8
Here
Iteration: 9
Here
Iteration: 10
Here
Iteration: 11
Here
Iteration: 12
Here
Iteration: 13
Here
Iteration: 14
Here
Iteration: 15
Here
Iteration: 16
Here
Iteration: 17
Here
Iteration: 18
Here
Iteration: 19
Here
Iteration: 20
Here
Iteration: 21
Here
Iteration: 22
Here
Iteration: 23
Here
Iteration: 24
Here
Iteration: 25
Here
Iteration: 26
Here
Iteration: 27
Here
Iteration: 28
Here
Iteration: 29
Here
Iteration: 30
Here
Iteration: 31
Here
Iteration: 32
Here
Iteration: 33
Here
Iteration: 34
Here
Iteration: 35
Here
Iteration: 36
Here
Iteration: 37
Here
Iteration: 38
Here
Iteration: 39
Here
Iteration: 40
Here
Iteration: 41
Here
Iteration: 42
Here
Iteration: 43
Here
Iteration: 44
Here
Iteration: 45
Here
Iteration: 46
Here
Iteration: 47
Here
Iteration: 48
Here
Iteration: 49
Here
Iteration: 50
Here
Iteration: 51
Here
Iteration: 52
Here
Iteration:

In [ ]:
model.evaluate(
    x = model_generator(X_TEST_DATA_PATH, y_TEST_DATA_PATH, 4, TEST_DATASET_NO // 4),
    verbose = 2,
    steps = TEST_DATASET_NO // 4 - 1
)

Iteration: 1
Here
Iteration: 2
Here
Iteration: 3
Here
Iteration: 4
Here
Iteration: 5
Here
Iteration: 6
Here
Iteration: 7
Here
Iteration: 8
Here
Iteration: 9
Here
Iteration: 10
Here
Iteration: 11
Here
Iteration: 12
Here
Iteration: 13
Here
Iteration: 14
Here
Iteration: 15
Here
Iteration: 16
Here
Iteration: 17
Here
Iteration: 18
Here
Iteration: 19
Here
Iteration: 20
Here
Iteration: 21
Here
Iteration: 22
Here
Iteration: 23
Here
Iteration: 24
Here
Iteration: 25
Here
Iteration: 26
Here
Iteration: 27
Here
Iteration: 28
Here
Iteration: 29
Here
Iteration: 30
Here
Iteration: 31
Here
Iteration: 32
Here
Iteration: 33
Here
Iteration: 34
Here
Iteration: 35
Here
Iteration: 36
Here
Iteration: 37
Here
Iteration: 38
Here
Iteration: 39
Here
Iteration: 40
Here
Iteration: 41
Here
Iteration: 42
Here
Iteration: 43
Here
Iteration: 44
Here
Iteration: 45
Here
Iteration: 46
Here
Iteration: 47
Here
Iteration: 48
Here
Iteration: 49
Here
Iteration: 50
Here
Iteration: 51
Here
Iteration: 52
Here
Iteration: 53
Here
It

[0.5824998617172241, 0.25872093439102173]

In [ ]:
model.save('/content/drive/MyDrive/Colab Datasets/')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Datasets/assets


In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/Colab Datasets/')

In [ ]:
model.predict(
    x = batch_generator(X_PREDICT_DATA_PATH, 10),
    verbose = 2,
    steps = 1
)

1/1 - 0s - 50ms/epoch - 50ms/step


array([[ 0.4009863 ,  1.0765753 , -0.15926272, -0.19789624, -0.27891365,
        -0.23617809,  1.5195467 ,  1.5206347 ,  1.5058571 ,  1.543524  ],
       [ 0.4283042 ,  1.0662307 , -0.11507791, -0.11163039, -0.15153116,
        -0.23888867,  1.4907284 ,  1.466741  ,  1.5485963 ,  1.6165657 ],
       [ 0.42919406,  1.1072943 , -0.14019091, -0.15511933, -0.20430094,
        -0.14249021,  1.5735509 ,  1.4805515 ,  1.516886  ,  1.6409643 ],
       [ 0.39637917,  1.1404443 , -0.07107676, -0.05016877,  0.07926704,
        -0.1019162 ,  1.8620148 ,  1.5566443 ,  1.687626  ,  2.0497434 ],
       [ 0.3928473 ,  1.155786  , -0.17575395, -0.23291108, -0.16872147,
        -0.17400968,  1.546263  ,  1.4094223 ,  1.4252734 ,  1.5768317 ],
       [ 0.422849  ,  1.0968893 , -0.13878062, -0.14887094, -0.04577526,
        -0.08827326,  1.8586514 ,  1.5713136 ,  1.7391957 ,  1.9665766 ],
       [ 0.43058622,  1.1291766 , -0.15314339, -0.13505672, -0.13905883,
        -0.12465794,  1.6660717 ,  1.51633   